In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:

# 1. 데이터 준비
# scikit-learn에서 유방암 데이터셋 불러오기
cancer = load_breast_cancer()
X = cancer.data   # 30개의 특성
y = cancer.target # 0: 악성(Malignant), 1: 양성(Benign)


In [4]:
print(cancer.keys)

<built-in method keys of Bunch object at 0x316ccf170>


In [24]:


# 데이터 표준화
# 신경망 학습을 위해 각 특성의 평균을 0, 분산을 1로 만듭니다.
scaler = StandardScaler()
X = scaler.fit_transform(X)


# 학습(train) 데이터와 테스트(test) 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# NumPy 배열을 PyTorch 텐서로 변환
# y_train, y_test는 이진 분류이므로 float32로 변환 후 손실 함수에서 처리
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1) # 이진 분류를 위해 shape 변경
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [37]:



# 텐서 데이터셋 및 데이터로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)



In [25]:
# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
print(X.shape)

(569, 30)


In [32]:


# 2. 모델 정의 (다층 퍼셉트론, MLP)
# 유방암 데이터는 30개의 특성을 가지므로 입력 레이어는 30개 노드,
# 양성/악성 2가지로 분류하므로 출력 레이어는 1개 노드를 가집니다.
class CancerClassifier(nn.Module):
    def __init__(self):
        super(CancerClassifier, self).__init__()
        self.fc1 = nn.Linear(30, 64)  # 입력(30) -> 은닉층(64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)  # 은닉층(64) -> 은닉층(32)
        self.fc3 = nn.Linear(32, 1)   # 은닉층(32) -> 출력(1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x


In [33]:

# 모델, 손실 함수, 옵티마이저 초기화
model = CancerClassifier().to(device)
# 이진 분류를 위한 손실 함수: BCEWithLogitsLoss (Sigmoid + Binary Cross Entropy)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [43]:
# 3. 모델 학습
def train_model(epochs):
    model.train()  # 학습 모드
    for epoch in range(epochs):
        i=1
        for inputs, labels in train_loader: #batch_size만큼씩 불러온다 
            inputs, labels = inputs.to(device), labels.to(device)
            if i==1: 
                print(inputs)
                print(labels)
                i+=1
            optimizer.zero_grad()  # 그라디언트 초기화
            outputs = model(inputs) # 순전파
            loss = criterion(outputs, labels) # 손실 계산
            loss.backward() # 역전파
            optimizer.step() # 가중치 업데이트
        
#         # 에포크마다 손실 출력
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    print('학습 완료!')


In [44]:
def evaluate_model():
    model.eval()  # 평가 모드
    with torch.no_grad(): # 그라디언트 계산 비활성화
        correct = 0
        total = 0
        i=1
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs) #예측을 함 
           
            # Sigmoid 함수를 적용하여 0 또는 1로 분류
            predicted = torch.round(torch.sigmoid(outputs))
            if i==1:
                print("실제값", labels) #실제값
                print("예측치", outputs) #예측치
                print("예측지2", predicted)
                i+=1

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'테스트 데이터셋 정확도: {accuracy:.2f}%')

In [58]:
train_model(epochs=3)
evaluate_model()

tensor([[ 3.9713e+00, -1.9074e-01,  3.9761e+00,  5.2448e+00,  1.2696e+00,
          8.9563e-01,  2.9040e+00,  2.8523e+00, -5.9736e-01, -1.0700e+00,
          8.9069e+00,  4.7018e-01,  9.4620e+00,  1.0677e+01,  2.1364e+00,
          1.2530e-01,  1.0609e+00,  3.6884e-01,  3.3040e+00,  2.5763e-01,
          2.4520e+00, -1.1737e+00,  2.4198e+00,  2.8450e+00, -7.9644e-01,
         -6.5309e-01,  2.2986e-01,  6.8358e-01, -2.0267e+00, -1.5902e+00],
        [-8.7115e-01, -1.0075e+00, -8.4354e-01, -7.9916e-01,  4.9387e-01,
         -2.5340e-01, -4.3728e-01, -4.0004e-01, -1.3004e-01,  8.8918e-01,
         -8.8452e-01, -7.1549e-01, -7.4802e-01, -6.4987e-01,  1.2501e-01,
         -3.7491e-01, -1.4303e-01, -2.7513e-01,  3.5569e-01, -3.3207e-01,
         -9.0064e-01, -9.4078e-01, -8.1915e-01, -7.7451e-01,  4.1343e-01,
         -2.1161e-01, -2.0288e-01, -1.0820e-01,  6.4751e-01,  3.7983e-01],
        [-1.0472e+00, -8.9118e-01, -1.0441e+00, -9.2516e-01,  6.3620e-01,
         -5.1398e-01, -1.0315e+00, -

In [2]:
def isLeap(year):
    if year % 4 == 0 and year % 100 != 0 or  400 == 0:
        return 1
    return 0

In [3]:
X = []
y = []
for i in range(1,5000):
    X.append(i)
    y.append(isLeap(i))

In [4]:
X = np.array(X).reshape(-1,1)
y = np. array(y)

In [5]:
print(X.shape)
print(y.shape)

(4999, 1)
(4999,)


In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [7]:
scalar = StandardScaler()

In [8]:
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)


In [9]:
X_train_tensor = torch.tensor(X_train,dtype=torch.float32)
y_train_tensor = torch.tensor(y_train,dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test,dtype=torch.float32)
y_test_tensor = torch.tensor(y_test,dtype=torch.float32).unsqueeze(1)

In [21]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [10]:


# 2. 모델 정의 (다층 퍼셉트론, MLP)
# 유방암 데이터는 30개의 특성을 가지므로 입력 레이어는 30개 노드,
# 양성/악성 2가지로 분류하므로 출력 레이어는 1개 노드를 가집니다.
class LeapYearNet(nn.Module):
    def __init__(self,input_size=1):
        super(LeapYearNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # 입력(30) -> 은닉층(64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)  # 은닉층(64) -> 은닉층(32)
        self.fc3 = nn.Linear(32, 1)   # 은닉층(32) -> 출력(1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [16]:
# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:

# 모델, 손실 함수, 옵티마이저 초기화
model = LeapYearNet().to(device)
# 이진 분류를 위한 손실 함수: BCEWithLogitsLoss (Sigmoid + Binary Cross Entropy)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# 3. 모델 학습
def train_model(epochs):
    model.train()  # 학습 모드
    for epoch in range(epochs):
        i=1
        for inputs, labels in train_loader: #batch_size만큼씩 불러온다 
            inputs, labels = inputs.to(device), labels.to(device)
            if i==1: 
                print(inputs)
                print(labels)
                i+=1
            optimizer.zero_grad()  # 그라디언트 초기화
            outputs = model(inputs) # 순전파
            loss = criterion(outputs, labels) # 손실 계산
            loss.backward() # 역전파
            optimizer.step() # 가중치 업데이트
        
#         # 에포크마다 손실 출력
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    print('학습 완료!')


In [19]:
def evaluate_model():
    model.eval()  # 평가 모드
    with torch.no_grad(): # 그라디언트 계산 비활성화
        correct = 0
        total = 0
        i=1
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs) #예측을 함 
           
            # Sigmoid 함수를 적용하여 0 또는 1로 분류
            predicted = torch.round(torch.sigmoid(outputs))
            if i==1:
                print("실제값", labels) #실제값
                print("예측치", outputs) #예측치
                print("예측지2", predicted)
                i+=1

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'테스트 데이터셋 정확도: {accuracy:.2f}%')

In [22]:
train_model(epochs=3)
evaluate_model()

tensor([[ 0.3235],
        [ 0.1414],
        [ 1.0504],
        [-1.1491],
        [-1.4284],
        [-0.6682],
        [-0.2950],
        [-1.0400],
        [-0.8669],
        [ 0.0302],
        [-0.4117],
        [-1.4847],
        [-1.0990],
        [-1.6988],
        [ 0.4681],
        [-0.4944]])
tensor([[0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.]])
Epoch [1/3], Loss: 0.4890
tensor([[-1.5493],
        [-1.5570],
        [ 0.8107],
        [-0.1470],
        [-0.7279],
        [ 0.2915],
        [ 0.2380],
        [-0.1206],
        [-0.3749],
        [ 1.0059],
        [ 0.9107],
        [ 0.4583],
        [-0.8489],
        [-0.9253],
        [ 0.5285],
        [ 1.3478]])
tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [26]:
# 1. 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001  #학습률
epochs = 5 
#장치 gpu가 없으면 cpu에서 작동하라 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. 데이터 준비
# 이미지 전처리를 위한 변환(Transform) 정의
transform = transforms.Compose([
    transforms.ToTensor(),  # PIL 이미지를 PyTorch 텐서로 변환 (0~1 사이 값)
    transforms.Normalize((0.5,), (0.5,)) 
    # 텐서를 평균 0.5, 표준편차 0.5로 정규화
])

In [31]:
# MNIST 학습 데이터셋 다운로드 및 로드
train_dataset = datasets.CIFAR100(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# MNIST 테스트 데이터셋 다운로드 및 로드
test_dataset = datasets.CIFAR100(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [44]:
# 3. 모델 정의 (CNN)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1) #3
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)

       

        self.fc1 = nn.Linear(64 * 8 * 8, 256)  # 
        self.fc2 = nn.Linear(256, 128) #100 
        self.fc3 = nn.Linear(128, 100)  
 
    def forward(self, x):
        x = self.conv1(x) #3 32
        x = self.relu(x)
        x = self.maxpool(x) # 32 16 16 
        
        x = self.conv2(x) #64 16 16 
        x = self.relu(x)
        x = self.maxpool(x) # 64 8 8 
        x = x.view(x.size(0), -1)  # 4096

        x = self.fc1(x) 
        x = self.relu(x)
        x = self.fc2(x) 
        x = self.relu(x)
        x = self.fc3(x) 
        return x


In [45]:





# 모델, 손실 함수, 옵티마이저 초기화
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 4. 모델 학습
def train_model():
    model.train()  # 모델을 학습 모드로 설정
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            # 그라디언트 초기화
            optimizer.zero_grad()

            # 순전파 (Forward pass)
            outputs = model(images)
            loss = criterion(outputs, labels)

            # 역전파 및 가중치 업데이트
            loss.backward()
            optimizer.step()

            running_loss += loss.item() #내부에 저장된 단일 값(스칼라 값)을 표준 Python 숫자로 추출
            if (i+1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
                running_loss = 0.0
    print('학습 완료!')

# 5. 모델 평가
def evaluate_model():
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():  # 그라디언트 계산 비활성화
        correct = 0
        total = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'테스트 데이터셋 정확도: {accuracy:.2f}%')





In [46]:
train_model()
evaluate_model()

Epoch [1/5], Step [100/782], Loss: 4.4259
Epoch [1/5], Step [200/782], Loss: 3.9872
Epoch [1/5], Step [300/782], Loss: 3.7912
Epoch [1/5], Step [400/782], Loss: 3.6150
Epoch [1/5], Step [500/782], Loss: 3.5013
Epoch [1/5], Step [600/782], Loss: 3.3886
Epoch [1/5], Step [700/782], Loss: 3.2774
Epoch [2/5], Step [100/782], Loss: 3.0335
Epoch [2/5], Step [200/782], Loss: 2.9992
Epoch [2/5], Step [300/782], Loss: 2.9473
Epoch [2/5], Step [400/782], Loss: 2.9233
Epoch [2/5], Step [500/782], Loss: 2.8638
Epoch [2/5], Step [600/782], Loss: 2.8467
Epoch [2/5], Step [700/782], Loss: 2.7537
Epoch [3/5], Step [100/782], Loss: 2.5729
Epoch [3/5], Step [200/782], Loss: 2.5423
Epoch [3/5], Step [300/782], Loss: 2.5665
Epoch [3/5], Step [400/782], Loss: 2.5579
Epoch [3/5], Step [500/782], Loss: 2.5257
Epoch [3/5], Step [600/782], Loss: 2.5086
Epoch [3/5], Step [700/782], Loss: 2.5132
Epoch [4/5], Step [100/782], Loss: 2.3055
Epoch [4/5], Step [200/782], Loss: 2.3002
Epoch [4/5], Step [300/782], Loss: